In [22]:
import xarray as xr
import re
from collections import defaultdict

In [23]:
## Imports
# BK 1850 tags
bk = xr.open_dataset('/net/paleonas.wustl.edu/volume1/blkshare/ajthompson/postproc/' \
'f.e12.F_1850_CAM5.wiso.f19.0ka.002.watertags.2.cam.h0.0006-0025.climo.nc')

# aaf 2090 tags
aaf = xr.open_dataset('/RAID/datasets/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004/archive/atm/hist/climatology/' \
'f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004.cam.h0.2105-2124_monthly_climatology_cat.nc')

In [33]:
#######################################################################################################
# 
# This function combines the variables associated with a given list of tagged regions
#   to compare the input with other water tag experiments that use different tags.
# For example, combine Europe and Asia tags to compare with an experiment that uses a Eurasia tag.
# Note that the outer edges of the combined tags must be equal to the comparison target.
#
#######################################################################################################

import re
from collections import defaultdict
import xarray as xr

def combine_regions_to_new_tag(  # same name as before for easy swapping
    ds: xr.Dataset,
    regions=("EURO", "NASA", "INDA", "SASA"),
    new_region="ERAS",
    *,
    # Optional per-region multipliers applied before summing (e.g., scale by area fraction)
    weights: dict | None = None,     # {"EURO":1, "NASA":1, ...}
    require_all: bool = False,       # if True, only combine when *all* regions for a group are present
    keep_nonregion_vars: bool = True,
    dtype="float32",
    skipna=True,                     # True → ignore NaNs when summing
    join="exact",                    # "exact" or "outer" coord alignment
    zero_fill=False                  # if join!="exact": fill missing coords with 0 before summing
) -> xr.Dataset:
    """
    Collapse per-region variables into a new *summed* region (e.g., ERAS).

    Matches variables shaped like:
      [<prefix>][<sep>]<REGION><tail>
    where <prefix> is optional, <sep> is an optional underscore, <REGION>∈regions,
    and <tail> is the remaining suffix (e.g., '18OI', 'V', 'r', etc.).

    Examples:
      'NASA18OI'         -> prefix='',     sep='',  region='NASA', tail='18OI'
      'PRECRC_NASA18Or'  -> prefix='PRECRC', sep='_', region='NASA', tail='18Or'

    Output variable name:
      f"{prefix}{sep}{new_region}{tail}"

    NOTE: This version **sums** across regions (not average). If you previously used the
    averaging build, this is the only logic change.
    """
    if not regions:
        raise ValueError("Provide at least one region prefix in `regions`.")

    region_alt = "|".join(map(re.escape, regions))
    pat = re.compile(rf"^(?P<prefix>.*?)(?P<sep>_)?(?P<region>{region_alt})(?P<tail>.+)$")

    groups = defaultdict(dict)    # (prefix, sep, tail) -> {region: DataArray}
    region_vars = []

    for vname, da in ds.data_vars.items():
        m = pat.match(vname)
        if m:
            key = (m.group("prefix") or "", m.group("sep") or "", m.group("tail"))
            groups[key][m.group("region")] = da
            region_vars.append(vname)

    new_vars = {}
    missing_summary = {}

    for (prefix, sep, tail), reg_map in groups.items():
        present = sorted(reg_map.keys())
        if require_all and len(present) != len(regions):
            missing_summary[(prefix, sep, tail)] = sorted(set(regions) - set(present))
            continue

        # collect, respecting canonical region order
        das, wts = [], []
        for r in regions:
            if r in reg_map:
                das.append(reg_map[r])
                wts.append(1.0 if weights is None else float(weights.get(r, 0.0)))
            elif not require_all:
                continue

        if not das:
            continue

        # align
        das_aligned = xr.align(*das, join=join)
        if weights is None:
            arr = xr.concat(das_aligned, dim="__region__")
        else:
            arr = xr.concat([d * w for d, w in zip(das_aligned, wts)], dim="__region__")

        # optional zero fill when using outer joins
        if zero_fill and join != "exact":
            arr = arr.fillna(0)

        # ********** SUM instead of MEAN **********
        eras = arr.sum(dim="__region__", skipna=skipna)
        # *****************************************

        eras = eras.astype(dtype)
        new_name = f"{prefix}{sep}{new_region}{tail}"
        eras.name = new_name
        new_vars[new_name] = eras

    # build output
    parts = []
    if keep_nonregion_vars:
        keep = [v for v in ds.data_vars if v not in set(region_vars)]
        parts.append(ds[keep])
    else:
        parts.append(xr.Dataset(coords=ds.coords))

    parts.append(xr.Dataset(new_vars))
    ds_out = xr.merge(parts)
    ds_out.attrs = dict(ds.attrs)

    if missing_summary:
        ds_out.attrs[f"{new_region}_missing_regions_info"] = str(missing_summary)

    return ds_out

In [34]:
# Average for 
eras_ds = combine_regions_to_new_tag(
    aaf,
    regions=("EURO", "NASA", "INDA", "SASA"),
    new_region="ERAS",
    # weights={"EURO":1, "NASA":1, "INDA":1, "SASA":1},  # optional
    require_all=True,       # set True if you only want averages when *all* are present
    keep_nonregion_vars=False # keep other non-region variables
)

In [35]:
eras_ds

<xarray.Dataset>
Dimensions:          (lev: 30, ilev: 31, time: 12, lat: 96, lon: 144, slat: 95,
                      slon: 144)
Coordinates:
  * lev              (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev             (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time             (time) object 2114-08-02 12:00:00 ... 2115-07-02 12:00:00
  * lat              (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon              (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * slat             (slat) float64 -89.05 -87.16 -85.26 ... 85.26 87.16 89.05
  * slon             (slon) float64 -1.25 1.25 3.75 6.25 ... 351.2 353.8 356.2
Data variables: (12/22)
    ERAS18OI         (time, lev, lat, lon) float32 0.0 0.0 ... 1.249e-08
    ERAS18OL         (time, lev, lat, lon) float32 0.0 0.0 ... 1.749e-08
    ERAS18OR         (time, lev, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ERAS18OS         (time, lev, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ERAS18OV         (time, lev, lat, lon) float32 3.298e-07 ... 7.44e-05
    ERAS18Or         (time, lev, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...               ...
    PRECRL_ERAS18OR  (time, lat, lon) float32 6.824e-21 6.824e-21 ... 3.601e-12
    PRECRL_ERASR     (time, lat, lon) float32 7.398e-21 7.398e-21 ... 3.664e-12
    PRECSC_ERAS18Os  (time, lat, lon) float32 0.0 0.0 ... 6.837e-11 6.843e-11
    PRECSC_ERASs     (time, lat, lon) float32 0.0 0.0 ... 7.035e-11 7.041e-11
    PRECSL_ERAS18OS  (time, lat, lon) float32 2.296e-13 2.296e-13 ... 3.439e-10
    PRECSL_ERASS     (time, lat, lon) float32 2.44e-13 2.44e-13 ... 3.529e-10
Attributes:
    Conventions:      CF-1.0
    source:           CAM
    case:             f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004
    title:            UNSET
    logname:          aflaim
    host:             dec1057
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     b.ie12.BRCP85C5CN.f19_g16.LME.004.cam.i.2100-01-01-0000...
    topography_file:  /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/consisten...
    history:          Wed Aug 27 08:17:51 2025: ncrcat f.ie12.BRCP85C5CN.f19_...
    NCO:              netCDF Operators version 5.0.6 (Homepage = http://nco.s...